<a href="https://colab.research.google.com/github/YASSIRAMRAOUI/Text-Mining/blob/main/Atelier_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Téléchargement des ressources NLTK
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

class PlagiarismDetector:
    def __init__(self, corpus):
        self.corpus = corpus
        self.documents = self._flatten_corpus()
        self.stop_words_fr = set(stopwords.words('french'))

    def _flatten_corpus(self):
        """Aplatit le corpus en une liste de documents"""
        docs = []
        for i, question in enumerate(self.corpus):
            for j, doc in enumerate(question):
                docs.append({
                    'text': doc,
                    'question_id': i,
                    'doc_id': j,
                    'type': ['Original', 'Reformulation', 'Synonymie légère', 'Plagiat clair'][j]
                })
        return docs

    def preprocess_text(self, text):
        """Prétraitement du texte"""
        # Conversion en minuscules
        text = text.lower()
        # Suppression de la ponctuation
        text = re.sub(r'[^\w\s]', ' ', text)
        # Tokenization
        tokens = word_tokenize(text, language='french')
        # Suppression des stopwords et filtrage
        tokens = [token for token in tokens if token not in self.stop_words_fr and len(token) > 2]
        return tokens

    def get_preprocessed_docs(self):
        """Retourne les documents prétraités"""
        return [' '.join(self.preprocess_text(doc['text'])) for doc in self.documents]

    # 1. Représentation One-Hot Vector (OHV)
    def ohv_representation(self):
        """Représentation One-Hot Vector"""
        preprocessed_docs = self.get_preprocessed_docs()
        vectorizer = CountVectorizer(binary=True)
        ohv_matrix = vectorizer.fit_transform(preprocessed_docs)
        return ohv_matrix, vectorizer

    # 2. Représentation Bag-of-Words (BOW)
    def bow_representation(self):
        """Représentation Bag-of-Words"""
        preprocessed_docs = self.get_preprocessed_docs()
        vectorizer = CountVectorizer()
        bow_matrix = vectorizer.fit_transform(preprocessed_docs)
        return bow_matrix, vectorizer

    # 3. Représentation TF-IDF
    def tfidf_representation(self):
        """Représentation TF-IDF"""
        preprocessed_docs = self.get_preprocessed_docs()
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)
        return tfidf_matrix, vectorizer

    # 4. Représentation SVD (LSA)
    def svd_representation(self, n_components=10):
        """Représentation SVD (Latent Semantic Analysis)"""
        tfidf_matrix, vectorizer = self.tfidf_representation()
        svd = TruncatedSVD(n_components=n_components, random_state=42)
        svd_matrix = svd.fit_transform(tfidf_matrix)
        return svd_matrix, svd, vectorizer

    # 5. Représentation Word Embeddings simplifiée (sans gensim)
    def simple_embedding_representation(self):
        """Représentation par embeddings simplifiés utilisant TF-IDF"""
        preprocessed_docs = self.get_preprocessed_docs()
        vectorizer = TfidfVectorizer(max_features=100)
        embedding_matrix = vectorizer.fit_transform(preprocessed_docs)
        return embedding_matrix, vectorizer

    # 6. Représentation par moyenne de caractères (alternative)
    def char_level_representation(self):
        """Représentation au niveau des caractères"""
        docs = [doc['text'] for doc in self.documents]

        # Création d'un vocabulaire de caractères
        all_chars = set(''.join(docs))
        char_to_idx = {char: idx for idx, char in enumerate(all_chars)}

        # Matrice de fréquences de caractères
        char_matrix = np.zeros((len(docs), len(all_chars)))
        for i, doc in enumerate(docs):
            for char in doc.lower():
                if char in char_to_idx:
                    char_matrix[i, char_to_idx[char]] += 1

        return char_matrix, char_to_idx

    def calculate_similarity_matrix(self, vectors):
        """Calcule la matrice de similarité cosinus"""
        if hasattr(vectors, 'shape'):  # Si c'est une matrice sparse
            return cosine_similarity(vectors)
        else:
            return cosine_similarity(np.array(vectors))

    def find_most_similar(self, similarity_matrix, method_name):
        """Trouve les 3 documents les plus similaires pour chaque document original"""
        results = {}

        # Les documents originaux sont aux indices 0, 4, 8, 12
        original_indices = [0, 4, 8, 12]

        for orig_idx in original_indices:
            question_id = self.documents[orig_idx]['question_id']
            similarities = []

            for i, sim_score in enumerate(similarity_matrix[orig_idx]):
                if i != orig_idx and self.documents[i]['question_id'] == question_id:
                    similarities.append((i, sim_score, self.documents[i]['type']))

            # Tri par similarité décroissante
            similarities.sort(key=lambda x: x[1], reverse=True)

            # Garder les 3 plus similaires
            top_3 = similarities[:3]

            results[f"Question {question_id + 1}"] = {
                'method': method_name,
                'original': self.documents[orig_idx]['text'][:50] + "...",
                'most_similar': [
                    {
                        'doc_id': sim[0],
                        'type': sim[2],
                        'similarity_score': round(sim[1], 4),
                        'text': self.documents[sim[0]]['text'][:50] + "..."
                    } for sim in top_3
                ]
            }

        return results

    def evaluate_plagiarism_detection(self, results):
        """Évalue la performance de détection du plagiat - VERSION CORRIGÉE"""
        evaluation = {}

        for question, data in results.items():
            detected_types = [sim['type'] for sim in data['most_similar']]

            # SYSTÈME DE SCORING CORRIGÉ (max 3 points par question)
            score = 0
            for i, doc_type in enumerate(detected_types):
                if i == 0 and doc_type == 'Plagiat clair':
                    score += 3  # Plagiat clair correctement identifié comme plus similaire
                elif i == 1 and doc_type == 'Reformulation':
                    score += 2  # Reformulation en 2ème position
                elif i == 2 and doc_type == 'Synonymie légère':
                    score += 1  # Synonymie légère en 3ème position

            evaluation[question] = {
                'detected_types': detected_types,
                'detection_score': score,  # Maximum 3 par question, 12 au total
                'perfect_detection': (detected_types[0] == 'Plagiat clair' if detected_types else False),
                'expected_order': ['Plagiat clair', 'Reformulation', 'Synonymie légère']
            }

        return evaluation

    def run_all_methods(self):
        """Exécute toutes les méthodes et compare les résultats"""
        all_results = {}
        evaluations = {}

        # OHV
        print("Calcul des similarités avec OHV...")
        ohv_matrix, _ = self.ohv_representation()
        ohv_similarity = self.calculate_similarity_matrix(ohv_matrix)
        all_results['OHV'] = self.find_most_similar(ohv_similarity, 'OHV')

        # BOW
        print("Calcul des similarités avec BOW...")
        bow_matrix, _ = self.bow_representation()
        bow_similarity = self.calculate_similarity_matrix(bow_matrix)
        all_results['BOW'] = self.find_most_similar(bow_similarity, 'BOW')

        # TF-IDF
        print("Calcul des similarités avec TF-IDF...")
        tfidf_matrix, _ = self.tfidf_representation()
        tfidf_similarity = self.calculate_similarity_matrix(tfidf_matrix)
        all_results['TFIDF'] = self.find_most_similar(tfidf_similarity, 'TFIDF')

        # SVD
        print("Calcul des similarités avec SVD...")
        svd_matrix, _, _ = self.svd_representation()
        svd_similarity = self.calculate_similarity_matrix(svd_matrix)
        all_results['SVD'] = self.find_most_similar(svd_similarity, 'SVD')

        # Simple Embedding (alternative à Word2Vec)
        print("Calcul des similarités avec Simple Embedding...")
        simple_emb_matrix, _ = self.simple_embedding_representation()
        simple_emb_similarity = self.calculate_similarity_matrix(simple_emb_matrix)
        all_results['SimpleEmbedding'] = self.find_most_similar(simple_emb_similarity, 'SimpleEmbedding')

        # Character Level
        print("Calcul des similarités avec Character Level...")
        char_matrix, _ = self.char_level_representation()
        char_similarity = self.calculate_similarity_matrix(char_matrix)
        all_results['CharacterLevel'] = self.find_most_similar(char_similarity, 'CharacterLevel')

        # Évaluation des performances
        for method, results in all_results.items():
            evaluations[method] = self.evaluate_plagiarism_detection(results)

        return all_results, evaluations

    def print_detailed_analysis(self, all_results, evaluations):
        """Affiche une analyse détaillée des résultats avec scoring corrigé"""

        print("=" * 100)
        print("ANALYSE DÉTAILLÉE DE LA DÉTECTION DE PLAGIAT")
        print("=" * 100)
        print("SYSTÈME DE SCORING CORRIGÉ:")
        print("- Plagiat clair en 1ère position = 3 points")
        print("- Reformulation en 2ème position = 2 points")
        print("- Synonymie légère en 3ème position = 1 point")
        print("- Maximum = 3 points par question, 12 points au total")
        print("=" * 100)

        # Pour chaque méthode
        methods = ['OHV', 'BOW', 'TFIDF', 'SVD', 'SimpleEmbedding', 'CharacterLevel']

        for method in methods:
            if method in all_results:
                print(f"\n{'='*50}")
                print(f"MÉTHODE: {method}")
                print(f"{'='*50}")

                results = all_results[method]
                evaluation = evaluations[method]

                total_score = 0
                perfect_detections = 0

                for q in range(1, 5):
                    question_key = f"Question {q}"
                    if question_key in results:
                        data = results[question_key]
                        eval_data = evaluation[question_key]

                        print(f"\n{question_key}:")
                        print(f"Original: {data['original']}")
                        print("Classement obtenu vs Attendu:")

                        # Afficher le classement avec comparaison
                        for i, sim in enumerate(data['most_similar']):
                            expected_type = eval_data['expected_order'][i] if i < 3 else "N/A"
                            match = "✓" if sim['type'] == expected_type else "✗"
                            print(f"  {i+1}. [{sim['type']:15}] Score: {sim['similarity_score']:.4f} {match} Attendu: {expected_type}")

                        print(f"Score: {eval_data['detection_score']}/3")
                        if eval_data['perfect_detection']:
                            print("🎯 DÉTECTION PARFAITE!")
                            perfect_detections += 1

                        total_score += eval_data['detection_score']

                print(f"\nRÉSUMÉ {method}:")
                print(f"Score total: {total_score}/12")
                print(f"Détections parfaites: {perfect_detections}/4")
                print(f"Performance: {total_score/12*100:.1f}%")

    def print_comparison_table(self, evaluations):
        """Affiche un tableau comparatif des méthodes avec scoring corrigé"""

        print("\n" + "=" * 80)
        print("TABLEAU COMPARATIF DES PERFORMANCES")
        print("=" * 80)

        comparison_data = []
        methods = ['OHV', 'BOW', 'TFIDF', 'SVD', 'SimpleEmbedding', 'CharacterLevel']

        for method in methods:
            if method in evaluations:
                total_score = 0
                perfect_detections = 0
                question_scores = []

                for q in range(1, 5):
                    question_key = f"Question {q}"
                    if question_key in evaluations[method]:
                        eval_data = evaluations[method][question_key]
                        total_score += eval_data['detection_score']
                        question_scores.append(eval_data['detection_score'])
                        if eval_data['perfect_detection']:
                            perfect_detections += 1

                comparison_data.append({
                    'Méthode': method,
                    'Score Total': total_score,
                    'Score Moyen': total_score / 4,
                    'Détections Parfaites': perfect_detections,
                    'Performance (%)': total_score / 12 * 100,
                    'Scores par Question': str(question_scores)
                })

        df_comparison = pd.DataFrame(comparison_data)
        df_comparison = df_comparison.sort_values('Score Total', ascending=False)
        print(df_comparison.round(2).to_string(index=False))

        # Meilleure méthode
        best_method = df_comparison.iloc[0]
        print(f"\n🏆 MEILLEURE MÉTHODE: {best_method['Méthode']}")
        print(f"   Score: {best_method['Score Total']}/12 - Performance: {best_method['Performance (%)']:.1f}%")

    def analyze_similarity_patterns(self, all_results):
        """Analyse les patterns de similarité entre les méthodes"""

        print("\n" + "=" * 80)
        print("ANALYSE DES PATTERNS DE SIMILARITÉ")
        print("=" * 80)

        methods = ['OHV', 'BOW', 'TFIDF', 'SVD', 'SimpleEmbedding', 'CharacterLevel']

        # Analyse par type de plagiat
        similarity_patterns = {
            'Plagiat clair': [],
            'Reformulation': [],
            'Synonymie légère': []
        }

        for method in methods:
            if method in all_results:
                results = all_results[method]

                for q in range(1, 5):
                    question_key = f"Question {q}"
                    if question_key in results:
                        data = results[question_key]

                        for sim in data['most_similar']:
                            similarity_patterns[sim['type']].append({
                                'method': method,
                                'question': q,
                                'score': sim['similarity_score']
                            })

        # Calcul des moyennes par type
        print("\nMOYENNES DE SIMILARITÉ PAR TYPE DE PLAGIAT:")
        print("-" * 50)

        for plagiat_type, scores in similarity_patterns.items():
            if scores:
                avg_score = np.mean([s['score'] for s in scores])
                min_score = np.min([s['score'] for s in scores])
                max_score = np.max([s['score'] for s in scores])
                print(f"{plagiat_type:20}: Moyenne = {avg_score:.4f} | Min = {min_score:.4f} | Max = {max_score:.4f}")

# Exécution du système
def main():
    # Corpus fourni
    Question1 = [
        "L'intelligence artificielle est un domaine de l'informatique qui vise à créer des systèmes capables de réaliser des tâches nécessitant habituellement l'intelligence humaine.",  # Original
        "L'IA est une branche de l'informatique dont l'objectif est de concevoir des machines capables d'exécuter des tâches typiquement humaines.",  # Reformulation
        "L'intelligence artificielle correspond à une discipline informatique cherchant à développer des systèmes pouvant accomplir des activités requérant l'intellect humain.",  # Synonymie légère
        "L'intelligence artificielle est un domaine de l'informatique qui vise à créer des systèmes capables de réaliser des tâches nécessitant habituellement l'intelligence humaine."  # Plagiat clair
    ]

    Question2 = [
        "L'apprentissage automatique est une sous-discipline de l'IA qui permet aux ordinateurs d'apprendre à partir de données sans être explicitement programmés.",  # Original
        "Le machine learning est une partie de l'intelligence artificielle où les systèmes informatiques apprennent des données sans instructions directes.",  # Reformulation
        "L'apprentissage automatique constitue un domaine de l'IA qui autorise les machines à s'entraîner sur des données sans programmation explicite.",  # Synonymie légère
        "L'apprentissage automatique est une sous-discipline de l'IA qui permet aux ordinateurs d'apprendre à partir de données sans être explicitement programmés."  # Plagiat clair
    ]

    Question3 = [
        "Le deep learning est une approche de l'apprentissage automatique qui utilise des réseaux de neurones profonds pour modéliser des données complexes.",  # Original
        "Le deep learning est une méthode du machine learning qui exploite des réseaux neuronaux à plusieurs couches pour analyser des données compliquées.",  # Reformulation
        "L'apprentissage profond est une technique d'IA qui emploie des architectures neuronales profondes afin de traiter des ensembles de données sophistiqués.",  # Synonymie légère
        "Le deep learning est une approche de l'apprentissage automatique qui utilise des réseaux de neurones profonds pour modéliser des données complexes."  # Plagiat clair
    ]

    Question4 = [
        "Un agent intelligent est un système qui perçoit son environnement et agit de manière autonome afin d'atteindre des objectifs spécifiques.",  # Original
        "Un agent intelligent correspond à un programme capable d'observer son environnement et de prendre des décisions indépendantes pour remplir une mission.",  # Reformulation
        "Un agent intelligent désigne un système qui détecte son contexte et agit automatiquement pour atteindre certains buts.",  # Synonymie légère
        "Un agent intelligent est un système qui perçoit son environnement et agit de manière autonome afin d'atteindre des objectifs spécifiques."  # Plagiat clair
    ]

    Corpus = [Question1, Question2, Question3, Question4]

    # Initialisation et exécution du détecteur
    print("Initialisation du détecteur de plagiat...")
    detector = PlagiarismDetector(Corpus)

    print("Prétraitement des documents...")
    preprocessed_docs = detector.get_preprocessed_docs()
    print(f"Nombre total de documents: {len(preprocessed_docs)}")
    print(f"Exemple de document prétraité: {preprocessed_docs[0][:60]}...")

    print("\nExécution de toutes les méthodes de détection...")
    print("Cette opération peut prendre quelques secondes...")

    all_results, evaluations = detector.run_all_methods()

    # Affichage des résultats
    detector.print_detailed_analysis(all_results, evaluations)
    detector.print_comparison_table(evaluations)
    detector.analyze_similarity_patterns(all_results)

    print("\n" + "=" * 80)
    print("CONCLUSION GÉNÉRALE")
    print("=" * 80)
    print("📊 RÉSUMÉ DES PERFORMANCES:")
    print("• Toutes les méthodes détectent parfaitement le plagiat clair (copie intégrale)")
    print("• Les méthodes diffèrent dans leur capacité à classer correctement reformulation et synonymie")
    print("• Le score maximum théorique est de 12 points (3 points × 4 questions)")
    print("• Les méthodes basées sur TF-IDF et SVD offrent généralement la meilleure granularité")
    print("• La méthode Character Level est très sensible mais moins discriminante")

if __name__ == "__main__":
    main()

Initialisation du détecteur de plagiat...
Prétraitement des documents...
Nombre total de documents: 16
Exemple de document prétraité: intelligence artificielle domaine informatique vise créer sy...

Exécution de toutes les méthodes de détection...
Cette opération peut prendre quelques secondes...
Calcul des similarités avec OHV...
Calcul des similarités avec BOW...
Calcul des similarités avec TF-IDF...
Calcul des similarités avec SVD...
Calcul des similarités avec Simple Embedding...
Calcul des similarités avec Character Level...
ANALYSE DÉTAILLÉE DE LA DÉTECTION DE PLAGIAT
SYSTÈME DE SCORING CORRIGÉ:
- Plagiat clair en 1ère position = 3 points
- Reformulation en 2ème position = 2 points
- Synonymie légère en 3ème position = 1 point
- Maximum = 3 points par question, 12 points au total

MÉTHODE: OHV

Question 1:
Original: L'intelligence artificielle est un domaine de l'in...
Classement obtenu vs Attendu:
  1. [Plagiat clair  ] Score: 1.0000 ✓ Attendu: Plagiat clair
  2. [Synonymie légè